# Predicting Federal Funds Rate Changes
    Xander de la Bruere
    Jakob Gertsch
    Sam Layton
    Matt Mella
    Wilson Stoddard

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import date, timedelta
import fredapi as fd
from prince import FAMD
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import LinearSegmentedColormap
from sklearn import linear_model

### Overview

The Federal Reserve (Fed) is a United States governmental institution that serves to keep the national economy stable and under control. By far the most powerful and influntial economic organization in the nation, the Fed is tasked with promoting "maximum employment, stable prices, and moderate long-term interest rates" [1]. Significantly, while this purpose statement contains three distinct items, the Fed's mission is commonly referred to as the "dual mandate". This is because moderating long-term interest rates actually forms the mechanism by which employment is maximized and inflation is curved.

In particular, since long-term interest rates essentially represent the cost of borowing money, a change in the prevailing interest rate significantly impacts levels of personal expenditures as well as corporate investment.  For example, when interest rates are relatively high, consumers have greater incentive to save money (and earn interest) rather than spend money, while companies have less incentive to invest (since they must pay a higher interest rate on their loans). This leads to an overall cooling in the economy. Conversely, lower interest rates increase personal consumption and corporate investment, causing the economy to heat up.

Long-term interest rates also have important implications for personal and institutional investment. For example, since bond prices are known to have an inverse relationship with bond prices, portfolios that are heavy in bonds lose value quickly when interest rates spike, leading to a painful loss in returns for investors [2]. In addition, even equity portfolios are negatively impacted by higher interest rates; since companies will spend less on new projects when the cost of borrowing is high, corporate profits taper, resulting in lower stock prices.

In the United States, the most important long-term interest rate is the federal funds rate (FFR), a target interest rate set by the Fed. When the Fed sets a new FFR, the Fed moves the prevailing interest rate observed in the economy towards this target interest rate through means such as requiring banks to keep more or less money in reserve or adjusting the number of federal bonds available on the market (thus increasing or decreasing the supply of money available for borrowing) [3]. As a result, the FFR almost always becomes the prevailing interest rate in the economy. Thus the FFR, through its effect on each of the personal and corporate spending and investment decisions discussed previously, has dramatic effects on a myriad of important economic indicators, including inflation and unemployment.

### Research Objectives

Due to the power that the FFR wields in the U.S. economy, the ability to predict changes in the FFR would allow consumers, corporations, and investment managers to make optimal financial decisions. For this reason, much effort has been expended to predict changes in the FFR. The Fed has shared some indicators that it considers in its decisions, including various price indices and labor market indicators, such as the annual change in the price index for personal consumption expenditures [1]. However, it does not explicitly share the way it considers or weights these metrics. Other groups that have attempted prediction, such as the Blue Chip consensus, are relatively successful at predicting changes in the FFR when most or all economic indicators are optimal (or adverse), but are inconsistent in accuracy when economic conditions are mixed [4]. We seek to build upon this work.

Our objective here is to use the techniques of regression and classification to develop a more robust model for predicting changes in the FFR. We are also concerned with finding features that best predict FOMC decisions to raise, lower, or hold constant the FFR. Since the naive prediction method of always predicting no change (the most common outcome) yields an accuracy score of 68% historically, we seek to develop models and methods that outperform this baseline in particular.

In short, we seek to answer these three key questions: 1) What features have the most predictive power for forecasting FOMC decisions? 2) What regression and classification methods are best suited to forecasting these decisions? 3) How accurately can we predict FOMC decisions?

### Our Data

In our search for data on key economic indicators that might influence the FFR, we looked to the St. Louis Federal Reserve (FRED) website [5]. This site hosts an enormous quantity of time series data on economic indicators. This data has been carefully and methodically tracked, and is a gold standard for economic data. Many reputable news sources, including the Wall Street Journal, frequently cite FRED data. In selecting features, we focused on features with data continuously tracked since 1989 (the year our FOMC decisions data starts) to the present day. We also chose features that we felt were most likely to have predictive power in determining the FOMC's decision to raise or lower interest rate. With these two considerations in mind, we selected the several features which are summarized in the table below.

| Feature | Variable | Frequency | Description |
| --- | --- | --- | --- |
| Bank Prime Loan Rate Changes | `loan` | daily | Rate charged by banks for short-term loans to creditworthy debtors |
| Exports of Goods and Services | `export` | quarterly | Total dollar value of goods and services exported in the quarter |
| Personal Consumption Expenditures Rate | `pce` | monthly | Measure of core inflation for personal expenditures |
| Unemployment Rate | `ue` | monthly | Number of unemployed as a percentage of the labor force |
| Change in Real GDP | `rgdp` | quarterly | Quarterly change in inflation-adjusted GDP |
| Total Vehicle Sales | `cars` | monthly | Total number of vehicle sales in millions |
| Recession Indicator | `recess` | monthly | Binary variable indicating whether the US is in a recession |
| GDP Deflator | `gdpd` | daily | Price index given by ratio of nominal GDP to real GDP |
| Velocity of M1 Money | `veloc` | quarterly | Ratio of nominal GDP to the quarterly average of M1 money stock |
| New Private Housing Units Started | `house` | monthly | Number of new housing units beginning construction in millions  |

In addition, we hypothesized that stock market performance and the strength of the dollar might be important factors in the FOMC's decision. The stock market acts as a barometer for the health of the economy, and the strength of the dollar indicates the strength of the economy relative to other countries. Thus, we used daily closing price data from Yahoo! Finance for the S&P 500 (`spx`, a proxy for total stock market) and US Dollar (`usd`, the value of the USD versus a basket of foreign currencies) Indices [6].

Considering the differences in opinion on economic policy that undoubtedly factor into the FOMC's decisions to raise or lower interest rates, we also generated binary variables for the political party of the Fed Chair (`fed_party`) and President of the United States (`potus_party`) [6], [7]. These variables are equal to 1 when the position was held by a Republican and 0 when held by a Democrat. Since this data is undisputed and readily accessible from several sources, we used a dataset available on Wikipedia after double-checking its veracity.

Our final explanatory feature, `cli`, was pulled from The Organisation for Economic Co-operation and Development (OECD) [8]. This organization is a multinational group founded in 1961 for the purpose of stimulating economic progress and world trade [9]. Given its status as a United Nations observer, we are confident in the validity of its data. The composite leading indicator (CLI) is a statistic created to "provide early signals of turning points in business cycles". Hence, the Fed would likely care about proactively adjusting interest rates in response to changes in this indicator.

Finally, our data on the FFR (`ffr`) came from the research of an economics professor at Williams University. Kenneth Kuttner is a leading researcher in monetary policy, and the data we downloaded from his website was the backbone of a paper he published in the Journal of Finance (the top journal in academic finance) [10]. It contains information on the percentage point change (`change`) at each announcement and the direction of each change (`direction`). The direction is indicated by -1 for a decrease, 0 for no change, and 1 for an increase in the FFR.

Citations:

1. https://fred.stlouisfed.org/
2. https://en.wikipedia.org/wiki/Chair_of_the_Federal_Reserve
3. https://simple.wikipedia.org/wiki/List_of_presidents_of_the_United_States
4. https://data.oecd.org/leadind/composite-leading-indicator-cli.htm
5. https://en.wikipedia.org/wiki/OECD
6. https://econ.williams.edu/faculty-pages/research/

### Data Cleaning

The data is pulled from five primary sources: FRED, Yahoo! Finance, OECD, Wikipedia, and Dr. Kuttner's website. FRED and Yahoo! Finance data is easily pulled using their API by entering a specific data series identifier. The OECD data was manually downloaded as a CSV and loaded. Wikipedia data on political parties was inputted by hand and also loaded from CSV. The FFR data from Dr. Kuttner's research page was downloaded as a CSV and loaded using pandas. The details of this process, including merging the data with different frequences, is demonstrated and explained in the Appendix. Given, our data was pulled from publicly accessible websites, and we thus expect it to be free from any legal problems.

The datasets we obtained were well-maintained, containing no missing data or invalid values. However, our data did not agree in frequency. For example, some variables including GDP and exports are reported quarterly, while others such as personal expenditures and the composite leading indicator are reported monthly. Further, a couple features (stock closing price and dollar index) change daily.

In addition, we realized that using the current day's values to predict changes in the federal funds interest rate would result in "look-ahead" bias. For example, it is unrealistic to believe that the closing stock price at 4:00 PM in New York on a given day would have any effect on an FOMC decision in Washington D.C. made at noon earlier that day. More likely, today's decisions are influenced by data ranging from one to several days prior.

To resolve these issues, we first filled in all empty days between reports of monthly and quarterly reported variables with the value from the previous report so we had values for all features for each day. We then used our `collapse()` function, which took our forward filled dataframe of over 12000 rows, shifted the data by a few days (to avoid look-ahead bias), and returned an aggregated dataframe with just 275 rows corresponding to scheduled FOMC announcement days. We exclude unscheduled FOMC announcements because people do not know they are happening until after they have been announced. In addition, nearly every unscheduled FOMC meeting has led to a change in rates, as they are often held for emergency reasons, such as COVID-19. We are focused only on predicting the results of known FOMC meetings, as these are the only meetings anyone can even attempt to predict without a knowledge of the future, so we drop the unscheduled meetings from our dataframe.

We have 3 different types of features, and we handled each differently. The first type was categorical data. In our given window (e.g., 60 days prior to the announcement until a week before the announcement) we take the most current data point (seven days before the announcement) and use that datapoint as our categorical feature in our newly generated dataframe. For monthly data, we take the final datapoint in our window (seven days before, just like we did for categorical), but we also add a new delta feature, which is the change from the start of the window. This essentially doubles the number of features we have in our dataset; we now have both a point estimate and its trend for the period. If our dataframe was 12561 rows by 5 monthly features, our new dataframe would be 275 days by 10 features now.

Finally, the third type of data changes on a daily basis. We wanted to have a point estimate as well as a trend for this data, so we decided to run a linear regression within each of the windows for each feature. We then take the 275 linear regression models made for each feature and report the corresponding betas (trend) and intercepts (point estimate). All of this data engineering is performed in the `collapse()` function. Our hypothesis is that both point estimate and trend will have an important role to play in our predictions of the rate changes. This is because changes in the target interest rate are often due to changes in economic conditions. Our `collapse()` function aggregates all data in the appropriate window preceding the FOMC meeting, extracting both the point estimate and trend of all indicators, excluding categorical data. Categorical data is excluded because it changes only over longer time horizons.

Generating these extra features likely has high autocorrelation, and as such, would likely hurt our model without some sort of dimensionality reduction. We perform Factor Analysis of Mixed Data (FAMD, an analogue to PCA for both categorical and continuous data) in order to only highlight the most important variation of our collapsed dataset. We then perform classification and learning analysis on this collapsed and cleaned data.

### Robustness

The data cleaning and feature engineering code found above is flexible and simple to modify. One can easily add not only new features but also recent FOMC decisions. Adding new data series from FRED and Yahoo! Finance is the easiest modification. Simply add the identifying name of the feature as well as the series code for FRED or the ticker for Yahoo! Finance, and the for loops will handle the rest. Adding data from outside CSV files only requires adding the name of the new dataframe to the list when creating `df_features`. Thus, any number of new features (columns) from a variety of sources can be quickly added. By virtue of the `ffill()` function, any data pulled using the FRED or Yahoo! Finance API is properly extended to the last row of the dataframe, which corresponds to the most recent FOMC announcement we have data for.

New FOMC announcement data can be appended by modifying the ffr_clean.csv file. This is done by adding the new target Fed Funds Rate under the `ffr` column and the change in basis points under the `change` column. In addition, the `scheduled` column should be entered as 1 if the meeting was scheduled and 0 if it was a surprise. The columns `decision` and `mom` are calculated automatically. It is critical to note that new FOMC announcements should be added sequentially and even if the meeting was unscheduled in order to keep the data consistent. Modifying this data is more of a hassle than adding new features, but meetings generally only occur every six weeks and require the addition of only three data points to the CSV file. All things considered, this is an insignificant burden to adding more data points. In addition, when new data points are added, the data coming from FRED and Yahoo! Finance automatically pull from 1989 until the present day. It is the responsibility of the researcher to ensure any data coming from CSV files is still accurate when adding new FOMC announcement data, as this will not automatically refresh, but autofill from the most recent valid entry using the `ffill()` function.

### Data Visualization and Basic Analysis



Below we plot the original dataset and the collapsed dataset in two dimensions using the factorial axes generated by FAMD (the analogue to principal components from PCA) to reduce dimensionality while minimizing information loss. 

![title](df_factor_analysis_2.png)

We see on the left the data is noisy and contains many small clusters. On the right the data is cleaner while preserving the same basic structure of the original data. 

### Learning Algorithms and In-Depth Analysis

We use random forests and XGBoost for training our models, using the collapsed data and using FAMD to reducethe dimensionality of the data to only 4 principle components. We use grid search to find the best hyperparameters for the models.

In the end, we find that the best model is a random forest with the following hyperparameters:

 {'max_depth': 20,
  'max_features': 'log2',
  'min_samples_leaf': 4,
  'min_samples_split': 5,
  'n_estimators': 50})

  The accuracy matrix for this model is:
                precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       0.59      0.90      0.71        30
         2.0       0.67      0.30      0.41        20

    accuracy                           0.60        55
   macro avg       0.42      0.40      0.37        55
weighted avg       0.56      0.60      0.54        55

### Ethical Implications

We have attempted to develop a model for predicting changes in the federal funds rate based on available economic indicators. As mentioned previously, the ability to predict changes to this prevailing interest rate before FOMC announcement allows for making strategical decisions to minimize or eliminate losses. For example, a prior prediction of what will happen to interest rates might motivate an individual to move from a low-security job to a high-security job, choose to wait or to rush to get a loan, or sell and buy stocks prior to a big move in the stock market.

Our intent and methodology raise a number of ethical issues. First, attempting to obtain an advantage through  information about the future inherently leads to others who do not share that information facing a disadvantage. For example, someone who has highly credible evidence to believe that the Federal Reserve will decrease the federal funds rate has incentive to sell stocks while keeping this information private so that others do not push the stock price down. Yet by not sharing that evidence, such an individual puts other investors in a more difficult place. Such moral grounds are the basis for material nonpublic information laws that prohibit using, sharing, or selling information about future company decisions that will affect stock prices to turn a profit (#A). While the information obtained through analysis such as that performed in this portfolio is not nearly as certain or exclusive (as the work we have done is highly reproducible), it is still important to acknowledge that successful attempts to predict the market in order to profit will inevitably result in someone losing in a zero-sum, competitive financial world.

Another ethical consideration is the potential loss that could be incurred when too much trust is placed in the prediction of the change in the federal fund rate generated by a model such as ours. With so many factors being weighed and considered by the Federal Reserve, and with economic indicators being so numerous and complex, it is inconceivable that any model used to predict changes in the federal funds rate could have near-perfect accuracy. Those that do not understand the extreme uncertainty involved in both the inputs and outputs to the model are likely to pivot too far, opening themselves up to loss incurred when the actual Federal Reserve decision flies in the face of predictions. In making and sharing predictions, therefore, it is important that equal emphasis and care is placed to express the uncertainty of the outcome, as well as to acknowledge that despite warnings, many will take risks based on the predictions that may have severe economic consequences.

Finally, it is important to note that our aim and methodology of predicting economic outcomes, when implemented and replicated on a large scale, may actually influence the outcomes themselves. Several times throughout history, the mere expectation of economic outcomes or a lack of faith in economic indicators have created economic disasters. For instance, in the events of Black Monday on October 19, 1987, the Dow Jones Industrial Average fell twenty percent in one day. The initial trigger was nervousness about possible overvaluation of stocks and the decline of the dollar, but the crisis was intensified by panic due to a lack of information as to what was happening ("B"). This episode and many others demonstrate that expectations or predictions of what will happen, whether grounded or totally unfounded, can often become a self-fulfilling prophecy. Thus it is possible that initially spurious relationships between economic indicators and interest rates mistakenly included or even emphasized in a model like this one might actually generate an unintended (and perhaps undesired) causal relationship, especially when machine learning is involved. Thus care must be taken to critically analyze how features are included, removed, or split on in such models, especially when destructive self-fulfilling feedback loops can have dramatic financial effects on millions across the country.  

### Citations:

[1] “Monetary Policy: What Are Its Goals? How Does It Work?” Federal Reserve Board - Monetary Policy: What Are Its Goals? How Does It Work?, www.federalreserve.gov/monetarypolicy/monetary-policy-what-are-its-goals-how-does-it-work.htm.

[2] Dziubinski, Susan. “How to Invest Your Money as the Fed Raises Interest Rates.” Morningstar, Inc., 27 July 2022, www.morningstar.com/portfolios/how-invest-your-money-fed-raises-interest-rates. 

[3] “Federal Open Market Committee.” The Fed - Federal Open Market Committee, www.federalreserve.gov/monetarypolicy/fomc.htm.

[4] Gavin, William T, and Rachel J Mandal. “The Art of Predicting the Federal Reserve: St. Louis Fed.” Saint Louis Fed Eagle, Federal Reserve Bank of St. Louis, 9 Dec. 2021, www.stlouisfed.org/publications/regional-economist/july-2000/inside-the-briefcase-the-art-of-predicting-the-federal-reserve.

[5] Chen, James. “Material Nonpublic Information (MNPI)? Definition and Laws.” Investopedia, Investopedia, www.investopedia.com/terms/m/materialinsiderinformation.asp.

[6] Carlson, Mark A. “A brief history of the 1987 stock market crash with a discussion of the Federal Reserve Response.” Finance and Economics Discussion Series, vol. 2006, no. 13, 2006, pp. 1–24, https://doi.org/10.17016/feds.2007.13.

### Appendix

The appendix contains both our code to pull the data pull and code to clean the data.

In [ ]:
### Data Pulls

# specify what data to pull from FRED
fred = fd.Fred(api_key = 'bde7928ce1d3cc555b5d2fb725f0ef4b')
start_dt = '1/1/1989'
name_id = [('loan', 'PRIME'), ('exports', 'A020RL1Q158SBEA'), ('pce', 'PCETRIM12M159SFRBDAL'),
           ('ue', 'UNRATE'), ('rgdp', 'A191RL1Q225SBEA'), ('cars', 'TOTALSA'), ('recess', 'USREC'),
           ('gdpd', 'A191RI1Q225SBEA'), ('veloc', 'M1V'), ('house', 'HOUST'), ('mich', 'MICH')]
fred_data = []

for x in name_id:
    # get FRED series using the proper id
    df = pd.DataFrame(fred.get_series(x[1], observation_start=start_dt))
    # make column name identifiable
    df.rename(columns={0: x[0]}, inplace=True)
    fred_data.append(df)

# aggregate FRED data series
df_fred = pd.concat(fred_data, axis=1)

# specify what data to pull from Yahoo! Finance
tickers = [('spx', '^SPX'), ('usd', 'DX-Y.NYB')]
start_dt = '1989-01-01'
end_dt = str(date.today() - timedelta(1))
interval = '1d'
stock_data = []

# access data and display
for t in tickers:
    # get price data using ticker
    df = yf.download(t[1], start = start_dt, end = end_dt, interval= interval)
    # make column name identifiable
    df = df[['Adj Close']].rename(columns={'Adj Close': t[0]})
    stock_data.append(df)

# aggregate stock data
df_stock = pd.concat(stock_data, axis=1)

# read in cli data from csv file downloaded off OECD website
df_cli = pd.read_csv('cli.csv', index_col=0)
df_cli.index = pd.to_datetime(df_cli.index)

# read in manually gathered political party data
df_party = pd.read_csv('party.csv', index_col=0)
df_party.index = pd.to_datetime(df_party.index)

# create feature dataset
df_features = pd.concat([df_fred, df_stock, df_cli, df_party], axis=1)

# pull in FOMC data 
df_fomc = pd.read_csv('ffr_clean.csv', index_col=0)
df_fomc.index = pd.to_datetime(df_fomc.index)

# create decisions and momentum column
decision = df_fomc['change'].values.copy()
down, up = tuple([decision < 0]), tuple([decision > 0])
decision[down], decision[up] = -1., 1.

# create new columns
df_fomc['decision'] = decision
df_fomc['mom'] = decision

# consider only scheduled meetings
df_fomc = df_fomc.query('scheduled == 1')
df_fomc.drop('scheduled', axis=1, inplace=True)

# get start and end dates
start_dt = df_features.index[0]
end_dt = df_fomc.index[-1]

# get every day between start and end date
delta = end_dt - start_dt
daily = [start_dt + timedelta(days=i) for i in range(delta.days + 1)]

# make dataframe of dates
df_days = pd.DataFrame(daily)
df_days.index = pd.to_datetime(df_days.values[:,0])

# initialize master dataframe
df = pd.concat([df_days, df_fomc, df_features], axis=1)

# drop irrelevant columns and rows
df.drop(0, axis=1, inplace=True)
last_ix = df_days.index.get_loc(end_dt) + 1
df.drop(df.index[last_ix:], inplace=True)

# find start and stop columns
cols = df.columns.to_list()
start_ix, end_ix = cols.index('mom'), cols.index('fed_party')

# shift necessary columns
df_shift = df.iloc[:, start_ix:end_ix].copy()
df.iloc[:, start_ix:end_ix] = df_shift.shift()

# fill in NaN entries
df_fill = df.iloc[:, start_ix:].copy()
df.iloc[:, start_ix:] = df_fill.ffill()

# save master dataframe
df.to_csv('master_data.csv')
df.tail()

In [ ]:
### Data Cleaning

# define the columns
monthly = ['pce','ue','cars','house','cli','exports','rgdp','gdpd','veloc', 'ffr','mich']
labels = ['decision', 'ffr', 'change']
daily = ['spx','usd', 'loan']
categorical = ['fed_party','potus_party','recess', 'mom']

# define the function to collapse the data to the event days
def collapse(df, key = 'label_day', start = 7, end = 60, cat_col = [], daily_col = [], other = [], labels = []):
    """A function to collapse the data into a single row for each event, capturing trends and point in time data.
    df: dataframe
    key: column name of the event day
    start: number of days before the event to capture
    end: number of days after the event to capture"""
    
    # Get our different dataframes
    categorical = df[cat_col]
    daily = df[daily_col]
    monthly = df[other]
    keep = df[key]
    labels = df[labels]

    # create the windows
    event_days = df[df[key]].index

    ################## Handle Categorical and Label Data ##################
    cat_collapsed = categorical[keep]
    y_labels = labels[keep]

    ################## Handle Monthly Data ##################
    # Create event windows
    windows = []
    for day in event_days:
        # Calculate start and end days of the window
        start_day = day - pd.Timedelta(days=start)
        end_day = day - pd.Timedelta(days=end)

        # Ensure start_day is not before the start of the dataset
        if end_day < df.index[0]:
            end_day = df.index[0]

        windows.append((start_day, end_day))

    # Initialize a list to store aggregated data
    aggregated_data = []

    for (start_day, end_day) in windows:
        # Select data within the window
        window_data = monthly.loc[end_day:start_day]

        # Apply your custom aggregation function
        final_vals = window_data.loc[start_day]
        change = final_vals - window_data.iloc[0]
        # relabel the change columns to include a d_ prefix
        change = change.rename(lambda x: 'd_' + x)

        # concatenate the data horizontally
        aggregated_window = pd.concat([final_vals, change], axis=0)
        aggregated_data.append(aggregated_window)

    # Combine aggregated data
    aggregated_monthly = pd.concat(aggregated_data, axis=1).T
    aggregated_monthly.index = event_days

    ################## Handle Daily Data ##################
    # define the model
    model = linear_model.LinearRegression()
    daily_agg = []

    # Get the names of the features
    daily_feat = daily.columns
    b0_name = ['b0_' + feat for feat in daily_feat]
    b1_name = ['b1_' + feat for feat in daily_feat]

    for (start_day, end_day) in windows:
        # Select data within the window
        window_data = daily.loc[end_day:start_day]
        time = np.arange(len(window_data)).reshape(-1, 1)

        # Initialize lists to store betas for each feature
        beta0 = []
        beta1 = []

        # Loop through each feature and fit the model
        for feat in daily_feat:
            model.fit(time, window_data[feat])
            beta0.append(model.intercept_)
            beta1.append(model.coef_[0])

        # Create DataFrames for betas
        beta0_df = pd.DataFrame([beta0], columns=b0_name)
        beta1_df = pd.DataFrame([beta1], columns=b1_name)

        # concatenate the data horizontally
        aggregated_window = pd.concat([beta0_df, beta1_df], axis=1)
        daily_agg.append(aggregated_window)

    # Combine aggregated data
    aggregated_daily = pd.concat(daily_agg, axis=0)
    aggregated_daily.index = event_days

    ################## Combine Data ##################
    # Combine all data
    combined = pd.concat([cat_collapsed, aggregated_monthly, aggregated_daily], axis=1)
    return combined, y_labels